In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver, get_xpath
from method.ours.parse import parse_form
from method.ours.prompts import get_form_context
from method.ours.history import HistoryTable
from method.ours.constraints import Invalid
from method.ours.generation import (
    generate_constraints_for_input_groups,
    generate_value_for_input_group,
    generate_values_for_input_groups,
    fill_form_with_value_table,
    submit_form
)
from method.ours.feedback import (
    get_local_feedback,
    get_global_feedback
)

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

URL = 'https://www.babycenter.com/pregnancy-weight-gain-estimator'
#'https://www.budget.com/en/home'
# 'https://www.yelp.com/'
# 'https://www.aircanada.com/ca/en/aco/home.html'

In [4]:
def get_to_form(driver):
    driver.get(URL)
    
    '''
    # AC - Multi-city
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
    ).click()
    '''
    
    '''
    # AC - My Bookings
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmg-tab-button-mngBook'))
    ).click()'''
    
    '''
    # Budget - Reservation
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'PicLoc_value'))
    ).click()
    '''

In [33]:
def find_form():
    return driver.find_element(
        By.XPATH,
        '//BODY/DIV[1]/DIV[3]/DIV[1]/DIV[5]/DIV[1]/DIV[1]/ARTICLE[1]/DIV[2]/DIV[1]/DIV[1]/DIV[2]'
    )


def find_button():
    return form.find_elements(By.TAG_NAME, 'button')[-1]
    # return None

In [5]:
driver = create_driver(HEADLESS)
get_to_form(driver)

In [23]:
form = find_form()

In [7]:
history_table = HistoryTable(
    url=URL,
    xpath=get_xpath(driver, form)
)

In [8]:
for element in form.find_elements(By.TAG_NAME, 'input'):
    try:
        element.clear()
    except:
        pass

# Processing

In [9]:
html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
relation_graph, input_groups = parse_form(
    driver,
    form,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/100 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:10<00:00,  2.49it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


# Generation

In [10]:
value_table = generate_constraints_for_input_groups(input_groups)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('left-weightNow'))\n.toBeNumerical()\n.toBeTruthy()\n.toHaveCompareCondition('>', 0)",
        "role": "assistant"
      }
    }
  ],
  "created": 1689881541,
  "id": "chatcmpl-7eTfpu3AFKegdENkX012k84sdoCaA",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 23,
    "prompt_tokens": 580,
    "total_tokens": 603
  }
}
  Total running cost: $0.059
toBeNumerical()
toBeNumerical() <re.Match object; span=(0, 15), match='toBeNumerical()'>
toBeNumerical ['']
toBeTruthy()
toBeTruthy() <re.Match object; span=(0, 12), match='toBeTruthy()'>
toBeTruthy ['']
toHaveCompareCondition('>', 0)
toHaveCompareCondition('>', 0) <re.Match object; span=(0, 30), match="toHaveCompareCondition('>', 0)">
toHaveCompareCondition ["'>'", '0']
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a se

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('ft'))\n.toBeNumerical()\n.toHaveCompareCondition('>', 0)\n.toHaveCompareCondition('<', 9)",
        "role": "assistant"
      }
    }
  ],
  "created": 1689881549,
  "id": "chatcmpl-7eTfxaSkymEgNOSpkCbE4JYaBnBBZ",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 643,
    "total_tokens": 670
  }
}
  Total running cost: $0.122
toBeNumerical()
toBeNumerical() <re.Match object; span=(0, 15), match='toBeNumerical()'>
toBeNumerical ['']
toHaveCompareCondition('>', 0)
toHaveCompareCondition('>', 0) <re.Match object; span=(0, 30), match="toHaveCompareCondition('>', 0)">
toHaveCompareCondition ["'>'", '0']
toHaveCompareCondition('<', 9)
toHaveCompareCondition('<', 9) <re.Match object; span=(0, 30), match="toHaveCompareCondition('<', 9)">
toHaveCompareCondition ["'<'", '9']
  Prompt: [
  {
  

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('I\\'m carrying twins')).toBeTruthy()",
        "role": "assistant"
      }
    }
  ],
  "created": 1689881555,
  "id": "chatcmpl-7eTg3oBDAlfdRoQDlvx2YIOZ8OtZr",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 12,
    "prompt_tokens": 658,
    "total_tokens": 670
  }
}
  Total running cost: $0.164
toBeTruthy()
toBeTruthy() <re.Match object; span=(0, 12), match='toBeTruthy()'>
toBeTruthy ['']
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only gene

In [11]:
value_table = generate_values_for_input_groups(input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "165",
        "role": "assistant"
      }
    }
  ],
  "created": 1689881607,
  "id": "chatcmpl-7eTgt8cJK9VtoMPJwf8W4ViTtvXSq",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 287,
    "total_tokens": 288
  }
}
  Total running cost: $0.228
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, an

In [12]:
fill_form_with_value_table(driver, value_table, input_groups)

<input>0</input> at y: (501, 557), x: (202, 486)
120
<input>value is lb</input> at y: (536, 558), x: (487, 552)
120
<input>0</input> at y: (602, 658), x: (202, 486)
150
<input>value is lb</input> at y: (636, 658), x: (487, 552)
125
<input>0</input> at y: (702, 758), x: (202, 364)
165
<input>value is ft</input> at y: (736, 758), x: (364, 426)
5.6
<input>0</input> at y: (702, 758), x: (441, 514)
165
<input></input> at y: (778, 820), x: (191, 233)
true
<input>value is 4</input> at y: (874, 896), x: (202, 551)
25


In [36]:
submit_form(driver, input_groups=input_groups, explicit_submit=find_button())

In [14]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

In [15]:
history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Feedback

In [25]:
new_form = find_form()

In [26]:
new_relation_graph, new_input_groups = parse_form(
    driver,
    new_form,
    prev_relation_graph=relation_graph,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/103 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:09<00:00,  2.87it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [27]:
global_feedback = get_global_feedback(html, new_html, remove_form_children=True)

In [28]:
global_feedback

['Please enter a valid height']

In [29]:
value_table = generate_constraints_for_input_groups(
    new_input_groups,
    value_table,
    global_feedback
)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('left-weightNow'))\n.toBeNumerical()\n.toBeTruthy()\n.toHaveCompareCondition('>', 0)",
        "role": "assistant"
      }
    }
  ],
  "created": 1689881740,
  "id": "chatcmpl-7eTj2jJTaFOB57A5Nvx26X7kgs08d",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 23,
    "prompt_tokens": 645,
    "total_tokens": 668
  }
}
  Total running cost: $0.329
toBeNumerical()
toBeNumerical() <re.Match object; span=(0, 15), match='toBeNumerical()'>
toBeNumerical ['']
toBeTruthy()
toBeTruthy() <re.Match object; span=(0, 12), match='toBeTruthy()'>
toBeTruthy ['']
toHaveCompareCondition('>', 0)
toHaveCompareCondition('>', 0) <re.Match object; span=(0, 30), match="toHaveCompareCondition('>', 0)">
toHaveCompareCondition ["'>'", '0']
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a se

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('left-height'))\n.toBeNumerical()\n.toBeTruthy()\n.toHaveCompareCondition('>=', 0)\n.toHaveCompareCondition('<=', 300)\n.toBeEqual(field('right-height'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1689881745,
  "id": "chatcmpl-7eTj74GtyzYkcfPFuTRT7rgsMmWn4",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 40,
    "prompt_tokens": 693,
    "total_tokens": 733
  }
}
  Total running cost: $0.375
toBeNumerical()
toBeNumerical() <re.Match object; span=(0, 15), match='toBeNumerical()'>
toBeNumerical ['']
toBeTruthy()
toBeTruthy() <re.Match object; span=(0, 12), match='toBeTruthy()'>
toBeTruthy ['']
toHaveCompareCondition('>=', 0)
toHaveCompareCondition('>=', 0) <re.Match object; span=(0, 31), match="toHaveCompareCondition('>=', 0)">
toHaveCompareCondition ["'>='", '0']
toHaveCompareCondition('<

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('right-height'))\n.toBeNumerical()\n.toBeTruthy()\n.toHaveCompareCondition('>', 0)\n.toHaveCompareCondition('<=', 300)\n.toBeEqual(field('left-height'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1689881751,
  "id": "chatcmpl-7eTjDkvZxXScuceIwRnjlQk68rO7F",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 39,
    "prompt_tokens": 703,
    "total_tokens": 742
  }
}
  Total running cost: $0.422
toBeNumerical()
toBeNumerical() <re.Match object; span=(0, 15), match='toBeNumerical()'>
toBeNumerical ['']
toBeTruthy()
toBeTruthy() <re.Match object; span=(0, 12), match='toBeTruthy()'>
toBeTruthy ['']
toHaveCompareCondition('>', 0)
toHaveCompareCondition('>', 0) <re.Match object; span=(0, 30), match="toHaveCompareCondition('>', 0)">
toHaveCompareCondition ["'>'", '0']
toHaveCompareCondition('<=', 3

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('pregnancyWeek'))\n.toBeNumerical()\n.toHaveCompareCondition('>=', 1)\n.toHaveCompareCondition('<=', 42)\n.toBeTruthy()",
        "role": "assistant"
      }
    }
  ],
  "created": 1689881755,
  "id": "chatcmpl-7eTjHsoLasdZa0EHQKdL4W09ioZN4",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 35,
    "prompt_tokens": 721,
    "total_tokens": 756
  }
}
  Total running cost: $0.468
toBeNumerical()
toBeNumerical() <re.Match object; span=(0, 15), match='toBeNumerical()'>
toBeNumerical ['']
toHaveCompareCondition('>=', 1)
toHaveCompareCondition('>=', 1) <re.Match object; span=(0, 31), match="toHaveCompareCondition('>=', 1)">
toHaveCompareCondition ["'>='", '1']
toHaveCompareCondition('<=', 42)
toHaveCompareCondition('<=', 42) <re.Match object; span=(0, 32), match="toHaveCompareCondition('<=', 42)">
toHaveCompareCo

In [30]:
value_table = generate_values_for_input_groups(new_input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "175",
        "role": "assistant"
      }
    }
  ],
  "created": 1689881764,
  "id": "chatcmpl-7eTjQS49h3oqWKN7klB6s3mDMKlRM",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 289,
    "total_tokens": 290
  }
}
  Total running cost: $0.511
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, an

In [31]:
fill_form_with_value_table(driver, value_table, new_input_groups)

<input>0 , value is 120</input> at y: (501, 557), x: (202, 486)
150
<input>value is lb</input> at y: (536, 558), x: (487, 552)
120
<input>0 , value is 150</input> at y: (602, 658), x: (202, 486)
175
<input>value is lb</input> at y: (636, 658), x: (487, 552)
125
<input>0 , value is 165</input> at y: (702, 758), x: (202, 364)
175
<input>value is ft</input> at y: (736, 758), x: (364, 426)
5.6
<input>0 , value is 165</input> at y: (702, 758), x: (441, 514)
175
<input></input> at y: (796, 838), x: (191, 233)
true
<input>value is 4</input> at y: (892, 914), x: (202, 551)
25


In [35]:
submit_form(driver, input_groups=new_input_groups, explicit_submit=find_button)

In [25]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Constraint Negation

In [17]:
form_context = get_form_context(input_groups)

In [25]:
history_table.table = history_table.table[:1]

In [28]:
get_attr = lambda x, a, d: x[a] if a in x else d

for field_id, entry in value_table.entries.items():
    input_type = get_attr(entry.input_group.node.element.attrs, 'type', 'text')
    if input_type in ['radio', 'checkbox']:
        continue
    
    for i in range(len(entry.constraints) + 1):
        get_to_form(driver)
        
        value_table_copy = value_table.copy()
    
        if i == 0:
            value_table_copy.entries[field_id].constraints = [Invalid()]
        else:
            value_table_copy.entries[field_id].constraints = [value_table.entries[field_id].constraints[i - 1].copy()]
            value_table_copy.entries[field_id].constraints[0].flip_negative()


        input_group = value_table_copy.entries[field_id].input_group

        value_table_copy = generate_value_for_input_group(
            input_group,
            value_table_copy,
            form_context=form_context
        )

        fill_form_with_value_table(driver, value_table_copy, input_groups)

        # driver.find_element(By.TAG_NAME, 'body').click()
        submit_form(driver, input_groups=input_groups, explicit_submit=find_button())
        
        time.sleep(1)
        
        new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
        global_feedback = get_global_feedback(html, new_html)
        
        print(global_feedback)
        
        history_table.add(
            value_table_copy.get_values_dict(),
            str(type(value_table_copy.entries[field_id].constraints[0])),
            global_feedback,
            driver.current_url
        )
        
        if len(global_feedback) == 0:
            value_table.entries[field_id].constraints[i].approve()

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

...

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "string:XYZ",
        "role": "assistant"
      }
    }
  ],
  "created": 1689872502,
  "id": "chatcmpl-7eRK2bQnEP3BXhVtwTHerVzr5Egwr",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 3,
    "prompt_tokens": 3248,
    "total_tokens": 3251
  }
}
  Total running cost: $1.216
<input>Enter your pick-up location or zip code</input> at y: (341, 391), x: (268, 768)
123 Main Street, Los Angeles, CA 90012
<input>mm/dd/yyyy</input> at y: (341, 391), x: (773, 1013)
07/21/2023
<select></select> at y: (341, 391), x: (1018, 1268)
string:8:00 AM
unable to fill input //BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[2]/DIV[6]/DIV[1]/DIV[1]/DIV[3]/SELECT[1]
<input>Return to same location</input> at y: (425, 475), x: (268, 768)
Los Angeles International Airport
<input>mm/dd/yyyy</input> at y: (425, 475), x: (773, 1013)
07/2

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "string:GB",
        "role": "assistant"
      }
    }
  ],
  "created": 1689872512,
  "id": "chatcmpl-7eRKCvo3S1ydLSLEz2g3xfU93ULRk",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 3,
    "prompt_tokens": 3247,
    "total_tokens": 3250
  }
}
  Total running cost: $1.314
<input>Enter your pick-up location or zip code</input> at y: (341, 391), x: (268, 768)
123 Main Street, Los Angeles, CA 90012
<input>mm/dd/yyyy</input> at y: (341, 391), x: (773, 1013)
07/21/2023
<select></select> at y: (341, 391), x: (1018, 1268)
string:8:00 AM
unable to fill input //BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[2]/DIV[6]/DIV[1]/DIV[1]/DIV[3]/SELECT[1]
<input>Return to same location</input> at y: (425, 475), x: (268, 768)
Los Angeles International Airport
<input>mm/dd/yyyy</input> at y: (425, 475), x: (773, 1013)
07/22

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


In [20]:
driver.get(URL)
time.sleep(1)
base_text = driver.find_element(By.TAG_NAME, 'body').text

In [29]:
for i, v in enumerate(history_table.table['values']):
    jv = json.loads(v)
    for key, value in jv.items():
        if key not in history_table.table.columns:
            history_table.table[key] = None
        history_table.table.loc[i, key] = value

In [32]:
history_table.table.loc[4, 'feedback']

'This should be date formatFEEDBACK-DELIMITERThis should be date formatFEEDBACK-DELIMITERThis should be date format'

In [30]:
history_table.table

,values,variation_type,feedback,new_url,//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[2]/DIV[6]/DIV[1]/DIV[1]/DIV[1]/ANGUCOMPLETE-ALT[1]/DIV[1]/INPUT[1],//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[2]/DIV[6]/DIV[1]/DIV[1]/DIV[2]/INPUT[1],//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[2]/DIV[6]/DIV[1]/DIV[1]/DIV[3]/SELECT[1],//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[2]/DIV[6]/DIV[1]/DIV[4]/DIV[1]/DIV[1]/ANGUCOMPLETE-ALT[1]/DIV[1]/INPUT[1],//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[2]/DIV[6]/DIV[1]/DIV[4]/DIV[1]/DIV[2]/INPUT[1],//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[2]/DIV[6]/DIV[1]/DIV[4]/DIV[1]/DIV[3]/SELECT[1],//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[2]/DIV[6]/DIV[1]/DIV[4]/DIV[4]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/SELECT[1],//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[2]/DIV[6]/DIV[1]/DIV[4]/DIV[4]/DIV[2]/DIV[1]/DIV[1]/SELECT[1]
0,"{""//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1...",base,Close\nSelect a Nearby Location\n\n\n\n\nLabel...,https://www.budget.com/en/home,"123 Main Street, Los Angeles, CA 90012",07/21/2023,string:8:00 AM,Los Angeles International Airport,07/22/2023,string:8:00 AM,number:22,string:US
1,"{""//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1...",<class 'method.ours.constraints.concrete.Inval...,Please enter a validFEEDBACK-DELIMITERPlease e...,https://www.budget.com/en/home,1234567890,07/21/2023,string:8:00 AM,Los Angeles International Airport,07/22/2023,string:8:00 AM,number:22,string:US
2,"{""//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1...",<class 'method.ours.constraints.concrete.ToBeT...,Connection Failed or Some Error occurred.\n\n\...,https://www.budget.com/en/home,,07/21/2023,string:8:00 AM,Los Angeles International Airport,07/22/2023,string:8:00 AM,number:22,string:US
3,"{""//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1...",<class 'method.ours.constraints.concrete.ToHav...,Close\nSelect a Nearby Location\n\n\n\n\nLabel...,https://www.budget.com/en/home,The constraints seem to contradict the input f...,07/21/2023,string:8:00 AM,Los Angeles International Airport,07/22/2023,string:8:00 AM,number:22,string:US
4,"{""//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1...",<class 'method.ours.constraints.concrete.Inval...,This should be date formatFEEDBACK-DELIMITERTh...,https://www.budget.com/en/home,"123 Main Street, Los Angeles, CA 90012",13/45/2022,string:8:00 AM,Los Angeles International Airport,07/22/2023,string:8:00 AM,number:22,string:US
5,"{""//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1...",<class 'method.ours.constraints.concrete.ToBeT...,Please enter a validFEEDBACK-DELIMITERPlease e...,https://www.budget.com/en/home,"123 Main Street, Los Angeles, CA 90012",,string:8:00 AM,Los Angeles International Airport,07/22/2023,string:8:00 AM,number:22,string:US
6,"{""//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1...",<class 'method.ours.constraints.concrete.ToHav...,Connection Failed or Some Error occurred.\n\n\...,https://www.budget.com/en/home,"123 Main Street, Los Angeles, CA 90012",09/15/2021,string:8:00 AM,Los Angeles International Airport,07/22/2023,string:8:00 AM,number:22,string:US
7,"{""//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1...",<class 'method.ours.constraints.concrete.ToHav...,Close\nSelect a Nearby Location\n\n\n\n\n\n\n\...,https://www.budget.com/en/home,"123 Main Street, Los Angeles, CA 90012",07/19/2023,string:8:00 AM,Los Angeles International Airport,07/22/2023,string:8:00 AM,number:22,string:US
8,"{""//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1...",<class 'method.ours.constraints.concrete.ToHav...,Close\nSelect a Nearby Location\n\n\n\n\n\n\n\...,https://www.budget.com/en/home,"123 Main Street, Los Angeles, CA 90012",07/23/2023,string:8:00 AM,Los Angeles International Airport,07/22/2023,string:8:00 AM,number:22,string:US
9,"{""//BODY/DIV[4]/SECTION[1]/DIV[2]/DIV[1]/DIV[1...",<class 'method.ours.constraints.concrete.Inval...,Close\

In [49]:
history_table.table = history_table.table[
    history_table.table.feedback.str.len() < len(base_text)
]

In [31]:
set('FEEDBACK-DELIMITER'.join(history_table.table.feedback).split('FEEDBACK-DELIMITER'))

{"Close\nSelect a Nearby Location\n\n\n\n\n\n\n\n\n\n0\n\t\t\t\t\t\t\tCar Rental Locations Found\n\n\n\nThere are no locations for this city. Please enter a new search or click here to browse for a location.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPlease clarify your Budget location\n\n\nWe Can't find the location. Please select from matches below or enter a new search.\n\nPossible Matches",
 "Close\nSelect a Nearby Location\n\n\n\n\n\n\n\n\n\n43\n\t\t\t\t\t\t\tCar Rental Locations Found\n\n\n\n\n\n\n\n\n1\nUnion Station (Amtrak)\n\n\n\n0.5 miles away\n\nAddress: 800 N Alameda St, (Near Cesar Chavez Ave), Los Angeles, US\n\n\nPhone:\n\t\t\t\t\t\t\t\t\t\t(1) 213-617-2977\n\t\t\t\t\t\t\t\t\t\n\nLocation Type:\nCorporate\n\n\n\nHours of Operation: Mon - Fri 8:00 AM - 5:00 PM; Sat 8:00 AM - 1:00 PM\n\t\t\t\t\t\t\t\t\t\n\nSelect This Location\n\n\n\n\n\n\n2\nThe Bloc Shopping Center\n\n\n\n0.9 miles away\n\nAddress: 700 W 7th St - Ste A200, (Inside The Bloc Garage), Los Angeles, US\n\n\nPhone:\n

In [34]:
len(set('FEEDBACK-DELIMITER'.join(history_table.table.feedback).split('FEEDBACK-DELIMITER')))

7

In [23]:
history_table.to_test_case('aoc-multitrip.py')